Demonstrate generation of multiple tones

In [ ]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
# After running demo_00_init.ipynb we decide which bitfile we'd like to use
bitfile = "mkids_2x2_kidsim_v2"
scan = Scan.Scan(bitfile)

In [ ]:
# Initialize simulation chain
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()

# Get the kids chain
chain = scan.kidsChain

# Set the mixer of the analysis and synthesis chains to be the same
fMixer = 324
fMixerQ = chain.fq(fMixer)
_ = scan.set_mixer(fMixerQ)

In [ ]:
pfb = getattr(chain.soc, chain.synthesis.dict['chain']['pfb'])
N= pfb.dict['N']
chsel = getattr(chain.soc, chain.analysis.dict['chain']['chsel'])


In [ ]:
ntrans = np.zeros(N)
idxs = np.zeros(N)
for ch in range(N):
    ntrans[ch],bit = chsel.ch2tran(ch)
    idxs[ch] = chsel.ch2idx(ch)
plt.plot(ntrans,'+',label="ntrans")
plt.plot(idxs,'x',label='idxs')
plt.xlabel("Input Channel Number")
plt.ylabel("Value")
plt.title(bitfile)
plt.legend()

In [ ]:
fTone = 314
decimation = 2
g = 0.9

qout = 2
chain.analysis.qout(qout)
chain.synthesis.qout(qout)
freqs = np.array([fTone+12.34, fTone])
fis = 0.123*np.arange(len(freqs))
gs = g*np.ones(len(freqs))/len(freqs)
gs[0] /= 2
cgs = None
print(gs)
chain.set_tones(freqs,fis,gs,cgs, verbose=True)

In [ ]:
chain.enable_channels(verbose=True)

In [ ]:
xs = chain.get_xs() # The first read might have "old" data in the first few samples
xs = chain.get_xs()

In [ ]:
phi0 = np.angle(xs[0])
phi1 = np.angle(xs[1])
plt.plot(phi0, label="Tone 0")
plt.plot(phi1, label="Tone 1")
plt.xlabel("sample number")
plt.ylabel("phase (Radians)")
plt.legend()

In [ ]:
amp0 = np.abs(xs[0])
amp1 = np.abs(xs[1])
plt.plot(amp0, label="Tone 0")
plt.plot(amp1, label="Tone 1")
plt.xlabel("sample number")
plt.ylabel("amplitude")
plt.legend()